In [ ]:
pip install -q torch torchvision timm accelerate torchvision==0.15.2
pip install -q huggingface-hub gradio pytorch-grad-cam==1.4.6 matplotlib tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from torchvision import datasets, transforms
import os
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])
val_transform = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])

from torchvision.datasets import CIFAR10
root = "/content/data_cifar"
for split in ["train","val"]:
    os.makedirs(os.path.join(root, split, "class_0"), exist_ok=True)


import torchvision.utils as vutils
ds = CIFAR10(root="/content/cifar", train=True, download=True)
import PIL.Image as Image
for i in range(200):
    img, label = ds[i]
    cls = f"class_{label%2}"
    p = os.path.join(root, "train", cls, f"{i}.png")
    os.makedirs(os.path.dirname(p), exist_ok=True)
    img.save(p)
ds2 = CIFAR10(root="/content/cifar", train=False, download=True)
for i in range(100):
    img, label = ds2[i]
    cls = f"class_{label%2}"
    p = os.path.join(root, "val", cls, f"{i}.png")
    os.makedirs(os.path.dirname(p), exist_ok=True)
    img.save(p)

print("Quick test dataset ready at", root)


In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os

DATA_DIR = ""

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

train_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_transform)
val_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_transform)

batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

num_classes = len(train_ds.classes)
print("Classes:", train_ds.classes, "Num classes:", num_classes)


In [ ]:
import timm
import torch.nn as nn

def create_model(num_classes):
    model = timm.create_model('mobilenetv3_small_100', pretrained=False, num_classes=num_classes)
    return model

model = create_model(num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model)


In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import os

epochs = 8
lr = 1e-3
weight_decay = 1e-4

optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

save_dir = "/content/checkpoints"
os.makedirs(save_dir, exist_ok=True)

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pbar = tqdm(loader, desc="train")
    for imgs, labels in pbar:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        pbar.set_postfix(loss=running_loss/total, acc=correct/total)
    return running_loss/total, correct/total

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        pbar = tqdm(loader, desc="val")
        for imgs, labels in pbar:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            pbar.set_postfix(loss=running_loss/total, acc=correct/total)
    return running_loss/total, correct/total

best_acc = 0.0
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    print(f"Train: loss {train_loss:.4f}, acc {train_acc:.4f} | Val: loss {val_loss:.4f}, acc {val_acc:.4f}")
    # save checkpoint
    ckpt = os.path.join(save_dir, f"mobilenet_epoch{epoch+1}_valacc{val_acc:.4f}.pt")
    torch.save({'epoch': epoch+1, 'model_state_dict': model.state_dict(), 'val_acc': val_acc}, ckpt)
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_dir, "best_model.pth"))
    # simple LR schedule
    for g in optimizer.param_groups:
        g['lr'] *= 0.98


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image

# load best model
model.load_state_dict(torch.load("/content/checkpoints/best_model.pth", map_location=device))
model.eval()


img_path = next(iter(val_ds.samples))[0]
img = Image.open(img_path).convert("RGB")
img_resize = val_transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    logits = model(img_resize)
    probs = torch.nn.functional.softmax(logits, dim=1)[0].cpu().numpy()
    pred = probs.argmax()
print("Predicted:", train_ds.classes[pred], "Confidence:", float(probs[pred]))

# Grad-CAM
target_layer = model.get_classifier() if hasattr(model, 'get_classifier') else model.blocks[-1]
# safe pick for timm mobile net:
try:
    target_layer = model.conv_head
except:
    # fallback: use last features layer
    for name, module in reversed(list(model.named_modules())):
        if isinstance(module, torch.nn.Conv2d):
            target_layer = module
            break

cam = GradCAM(model=model, target_layers=[target_layer], use_cuda=(device.type=='cuda'))
rgb_img = np.array(img.resize((224,224))) / 255.0
input_tensor = preprocess_image(rgb_img, mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225))
grayscale_cam = cam(input_tensor=input_tensor, targets=None)[0]
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.imshow(img.resize((224,224)))
plt.title("Input")
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(visualization)
plt.title("Grad-CAM")
plt.axis('off')
plt.show()


In [ ]:
import torch

model_cpu = model.to('cpu').eval()

example = torch.randn(1, 3, 224, 224)

traced = torch.jit.trace(model_cpu, example)
torch.jit.save(traced, "/content/checkpoints/mobilenet_traced.pt")
print("Saved TorchScript traced model.")

# Apply dynamic quantization on the traced model (only works on torch.nn.Linear, LSTM etc. but will reduce size)
# To quantize the original model weights (state dict) use torch.quantization.quantize_dynamic
quantized_model = torch.quantization.quantize_dynamic(
    model_cpu, {torch.nn.Linear}, dtype=torch.qint8
)
torch.jit.save(torch.jit.trace(quantized_model, example), "/content/checkpoints/mobilenet_quantized_traced.pt")
print("Saved quantized TorchScript model.")


In [ ]:
# login
huggingface-cli login


In [ ]:
from huggingface_hub import HfApi, login, create_repo, upload_file

from huggingface_hub import login
login(token="")

api = HfApi()
repo_id = ""   # change
api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)

# Upload artifacts
api.upload_file(path_or_fileobj="/content/checkpoints/mobilenet_traced.pt",
                path_in_repo="mobilenet_traced.pt",
                repo_id=repo_id)
api.upload_file(path_or_fileobj="/content/checkpoints/mobilenet_quantized_traced.pt",
                path_in_repo="mobilenet_quantized_traced.pt",
                repo_id=repo_id)


api.create_repo(repo_id="", repo_type="space", exist_ok=True)
from huggingface_hub import Repository
repo = Repository(local_dir="/content/space_repo", clone_from="your-username/vision-demo")
repo.push_to_hub(commit_message="Initial Gradio demo")
